# Stat Arb Pairs\n

In [ ]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
np.random.seed(42)
x = np.cumsum(np.random.normal(0,0.5,800)) + 100
y = x*1.2 + np.random.normal(0,0.8,800)
df = pd.DataFrame({'x':x,'y':y})
X = np.vstack([df['x'], np.ones(len(df))]).T
beta, alpha = np.linalg.lstsq(X, df['y'], rcond=None)[0]
spread = df['y'] - (beta*df['x'] + alpha)
plt.plot(spread.values); plt.title('Spread'); plt.show()
